### Github Access Token

In [2]:
import os
from dotenv import load_dotenv
import requests
import csv
from datetime import datetime
import time

load_dotenv()

GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")
HEADERS = {"Authorization": f"token {GITHUB_ACCESS_TOKEN}"}

### Fetch Users

In [3]:
def fetch_all_users(location="Toronto", min_followers=100):
    url = "https://api.github.com/search/users"
    params = {
        "q": f"location:{location} followers:>{min_followers}",
        "per_page": 100,
        "page": 1
    }
    users = []

    while True:
        response = requests.get(url, headers=HEADERS, params=params)
        
        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        users.extend(data.get("items", []))

        if len(data.get("items", [])) == 0 or len(users) >= 1000:
            break

        params["page"] += 1

        time.sleep(1)  

    return users

In [4]:
def fetch_user_details(username):
    url = f"https://api.github.com/users/{username}"
    return requests.get(url, headers=HEADERS).json()


### Fetch Repos

In [5]:
def fetch_repositories(username, max_repos=500):
    url = f"https://api.github.com/users/{username}/repos"
    params = {"per_page": 100, "page": 1}
    repos = []
    while len(repos) < max_repos:
        response = requests.get(url, headers=HEADERS, params=params).json()
        if not response:
            break
        repos.extend(response)
        params["page"] += 1
    return repos[:max_repos]

### Cleaning

In [6]:
def clean_company_name(name):
    if not name:
        return ""
    name = name.strip()
    if name.startswith("@"):
        name = name[1:]
    return name.upper()


### Generating the CSV files

In [7]:
def write_users_csv(users):
    with open("users.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([
            "login", "name", "company", "location", "email", "hireable", "bio",
            "public_repos", "followers", "following", "created_at"
        ])
        for user in users:
            user_details = fetch_user_details(user['login'])
            writer.writerow([
                user_details.get("login", ""),
                user_details.get("name", ""),
                clean_company_name(user_details.get("company", "")),
                user_details.get("location", ""),
                user_details.get("email", ""),
                user_details.get("hireable", ""),
                user_details.get("bio", ""),
                user_details.get("public_repos", 0),
                user_details.get("followers", 0),
                user_details.get("following", 0),
                user_details.get("created_at", "")
            ])

In [8]:
def write_repositories_csv(users):
    with open("repositories.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([
            "login", "full_name", "created_at", "stargazers_count",
            "watchers_count", "language", "has_projects", "has_wiki", "license_name"
        ])
        for user in users:
            repos = fetch_repositories(user['login'])
            for repo in repos:
                writer.writerow([
                    user['login'],
                    repo.get("full_name", ""),
                    repo.get("created_at", ""),
                    repo.get("stargazers_count", 0),
                    repo.get("watchers_count", 0),
                    repo.get("language", ""),
                    repo.get("has_projects", False),
                    repo.get("has_wiki", False),
                    repo.get("license", {}).get("name", "") if repo.get("license") else ""
                ])

In [9]:
users = fetch_all_users()
write_users_csv(users)
write_repositories_csv(users)

Question 1

In [1]:
import duckdb

con = duckdb.connect()

In [11]:
query = "SELECT * FROM read_csv_auto('users.csv')"
con.execute(query).fetchdf()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,aneagoie,Andrei Neagoie,None,"Toronto, Canada",None,True,Senior Software Dev turned Instructor. Founder...,145,10279,1,2015-01-30 17:05:43
1,ZhangMYihua,Yihua Zhang,None,Toronto,yihuazhang2@gmail.com,NaN,Toronto Software Developer,143,5801,11,2015-01-18 00:01:02
2,susanli2016,Susan Li,None,Toronto Canada,None,NaN,Chief Data Scientist,34,4921,68,2016-11-28 04:22:39
3,thedaviddias,David Dias,KIJIJICA,"Toronto, Canada",None,NaN,💻 Passionate Front-End Dev & 🎨 UI/UX fan. Cont...,88,4546,303,2010-04-05 14:40:12
4,ange-yaghi,Ange Yaghi,None,Toronto,me@angeyaghi.com,NaN,C++ Developer,32,4023,11,2016-07-13 21:01:21
...,...,...,...,...,...,...,...,...,...,...,...
680,acode,Autocode Open Source,None,"San Francisco, CA and Toronto, ON",None,NaN,Open Source Contributions for Autocode (defunct),9,101,0,2021-03-11 07:34:55
681,gorkem-cetin,Görkem Çetin,COUNTLY,"Toronto, Canada",None,NaN,"Director at Countly, #1 open source mobile ana...",44,101,8,2012-05-20 16:20:29
682,rwhogg,"Bob ""Wombat"" Hogg",GOOGLE,"Toronto, Canada",rwhogg@google.com,NaN,"Dev-of-all-trades, University of Waterloo grad...",43,101,473,2012-09-18 21:29:26
683,andrewdamelio,Andrew D'Amelio,None,"Toronto, Canada",None,NaN,npx andrewdamelio,31,101,66,2012-04-27 01:39:33


In [12]:
query = "SELECT login from read_csv_auto('users.csv') ORDER BY followers DESC LIMIT 5"
con.execute(query).fetchdf()

,login
0,aneagoie
1,ZhangMYihua
2,susanli2016
3,thedaviddias
4,ange-yaghi


In [13]:
query = "SELECT login from read_csv_auto('users.csv') ORDER BY created_at ASC LIMIT 5"
con.execute(query).fetchdf()

,login
0,jamesmacaulay
1,michaelklishin
2,myles
3,nwjsmith
4,vito


In [14]:
query = "SELECT license_name from read_csv_auto('repositories.csv') WHERE license_name != '' GROUP BY license_name ORDER BY COUNT(*) DESC LIMIT 3"
con.execute(query).fetchdf()

,license_name
0,MIT License
1,Other
2,Apache License 2.0


In [15]:
query = "SELECT company, COUNT(*) as count from read_csv_auto('users.csv') WHERE company != '' GROUP BY company ORDER BY COUNT(*) DESC"
con.execute(query).fetchdf()

,company,count
0,UNIVERSITY OF TORONTO,21
1,SHOPIFY,16
2,NVIDIA,7
3,GOOGLE,7
4,YORK UNIVERSITY,5
...,...,...
304,DISCOVERYVIP,1
305,PROXET (FORMER RAILS REACTOR),1
306,VOLTRON DATA,1
307,CANOPY SECURITY,1


In [16]:
query = f"""
    SELECT language
    FROM read_csv_auto('repositories.csv') AS r
    JOIN read_csv_auto('users.csv') AS u ON r.login = u.login
    WHERE u.created_at > '2020-01-01' AND language != ''
    GROUP BY language
    ORDER BY COUNT(*) DESC
    LIMIT 1 OFFSET 1
"""
con.execute(query).fetchdf()

,language
0,TypeScript


In [18]:
query = f"""
    select language, avg(stargazers_count) as avg_stars
    from read_csv_auto('repositories.csv') as r
    group by language
    order by avg_stars desc
"""
con.execute(query).fetchdf()

,language,avg_stars
0,Cython,1780.833333
1,Forth,1191.000000
2,ASP.NET,414.000000
3,BrighterScript,313.000000
4,SAS,172.000000
...,...,...
209,J,0.000000
210,Web Ontology Language,0.000000
211,Meson,0.000000
212,PigLatin,0.000000


In [19]:
query = """
    select login, followers / (1 + following) as leader_strength
    from read_csv_auto('users.csv')
    order by leader_strength desc
    limit 5
"""
con.execute(query).fetchdf()

,login,leader_strength
0,aneagoie,5139.5
1,nayuki,3546.0
2,GrapheneOS,3524.0
3,hlissner,2424.0
4,rspivak,2180.0


In [20]:
query = f"""
    SELECT followers, public_repos
    FROM read_csv_auto('users.csv')
"""

toronto_users_df = con.execute(query).fetchdf()
correlation = toronto_users_df['followers'].corr(toronto_users_df['public_repos'])
print(correlation)

0.05549355862436628


In [21]:
import pandas as pd
import statsmodels.api as sm

In [22]:
X = toronto_users_df['public_repos']
y = toronto_users_df['followers']
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
slope = model.params['public_repos']

print(slope)

0.2528220678949066


In [2]:
query = "select * from read_csv_auto('repositories.csv')"
repos_df = con.execute(query).fetchdf()

In [5]:
print(repos_df['has_projects'])

0         True
1         True
2         True
3         True
4         True
         ...  
55269     True
55270     True
55271     True
55272    False
55273     True
Name: has_projects, Length: 55274, dtype: bool


In [8]:
correlation = (repos_df['has_projects'] == True).corr((repos_df['has_wiki'] == True))
print(correlation)

0.37284213193516674


In [25]:
query = "select * from read_csv_auto('users.csv')"
users_df = con.execute(query).fetchdf()

In [58]:
print(users_df)

             login             name                      company  \
0         aneagoie   Andrei Neagoie                         None   
1      ZhangMYihua      Yihua Zhang                         None   
2      susanli2016         Susan Li                         None   
3     thedaviddias       David Dias                     KIJIJICA   
4       ange-yaghi       Ange Yaghi                         None   
..             ...              ...                          ...   
95    nikolovlazar    Lazar Nikolov                    GETSENTRY   
96         mobinni         Mo Binni              ZERO TO MASTERY   
97  kulkarniankita  Ankita Kulkarni  HTTPS://FRONTENDSNACKS.DEV/   
98        ploopyco             None                       PLOOPY   
99       DylanVann       Dylan Vann                         None   

                      location                        email hireable  \
0              Toronto, Canada                         None     True   
1                      Toronto        y

In [26]:
import numpy as np

In [ ]:
avg_following_hireable = users_df[users_df['hireable'] == True]['following'].mean()
print(avg_following_hireable)
avg_following_non_hireable = users_df[users_df['hireable'] == False]['following'].mean()
print(avg_following_non_hireable)
difference = avg_following_hireable - avg_following_non_hireable
print(difference)

58.0
nan
nan


In [32]:
query = f"""
    SELECT bio, followers
    FROM read_csv_auto('users.csv')
    WHERE bio IS NOT NULL AND bio != ''
"""

users_df = con.execute(query).fetchdf()

users_df['bio_word_count'] = users_df['bio'].str.split().str.len()

X = users_df['bio_word_count']
y = users_df['followers']
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
slope = model.params['bio_word_count']

print(slope)

8.376736537822353


In [30]:
query = f"""
    SELECT login, created_at
    FROM read_csv_auto('repositories.csv')
"""

repos_df = con.execute(query).fetchdf()

repos_df['created_at'] = pd.to_datetime(repos_df['created_at'])

repos_df['is_weekend'] = repos_df['created_at'].dt.dayofweek >= 5

weekend_repos_count = repos_df[repos_df['is_weekend']].groupby('login').size().reset_index(name='count')

top_weekend_users = weekend_repos_count.sort_values(by='count', ascending=False).head(5)

top_users_logins = top_weekend_users['login'].tolist()

print("Top 5 users who created the most repositories on weekends (UTC):", ','.join(top_users_logins))

Top 5 users who created the most repositories on weekends (UTC): andyw8,QuinntyneBrown,n1ckfg,nhat416,rgrinberg


In [31]:
query = f"""
    SELECT name
    FROM read_csv_auto('users.csv')
    WHERE name IS NOT NULL AND name != ''
"""

users_df = con.execute(query).fetchdf()

users_df['surname'] = users_df['name'].str.strip().str.split().str[-1]

surname_counts = users_df['surname'].value_counts()

max_count = surname_counts.max()

most_common_surnames = surname_counts[surname_counts == max_count].index.tolist()

most_common_surnames.sort()

print("Most common surname(s):", ', '.join(most_common_surnames))

Most common surname(s): Ahmed
